In [ ]:
# https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/

In [10]:
#https://stackoverflow.com/questions/63201036/add-additional-layers-to-the-huggingface-transformers

In [1]:
#!pip install transformers

In [ ]:
from transformers import BertTokenizer, TFBertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
transformer_model = TFBertModel.from_pretrained('bert-base-uncased', output_hidden_states=False)

2021-11-01 08:06:18.118264: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-01 08:06:18.131671: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-01 08:06:18.132333: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-01 08:06:18.133575: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [2]:
import tensorflow as tf
input_ids = tf.keras.Input(shape=(768,), name='input_token', dtype='int32')
attention_mask = tf.keras.Input(shape=(768,), name='masked_token', dtype='int32')

embedding_layer = transformer_model([input_ids, attention_mask])[0]
cls_token = embedding_layer[:,0,:]

bert_encoder = tf.keras.models.Model(inputs = {"input_ids": input_ids, "attention_mask": attention_mask}, outputs = cls_token)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [4]:
def tokenize(sentences, max_length=768, padding='max_length'):
    """Tokenize using the Huggingface tokenizer
    Args:
        sentences: String or list of string to tokenize
        padding: Padding method ['do_not_pad'|'longest'|'max_length']
    """
    return tokenizer(
        sentences,
        truncation=True,
        padding=padding,
        max_length=max_length,
        return_tensors="tf"
    )

In [5]:
batch_sentences = ["Hello I'm a single sentence", "And another sentence", "And the very very last one"]

In [6]:
X = dict(tokenize(batch_sentences))

In [7]:
bert_encoder(X).shape

/opt/conda/lib/python3.9/site-packages/keras/engine/functional.py:582: UserWarning: Input dict contained keys ['token_type_ids'] which did not match any model input. They will be ignored by the model.
  warnings.warn(


TensorShape([3, 768])

In [24]:
###

In [11]:
data_set = 'app_store'

print('Starting to embedd %s data set ...' % data_set)

Starting to embedd app_store data set ...


In [12]:
import numpy as np
print('Load %s data set ...' % data_set)
data_X = np.load('./../data/datasets/%s_X.npy' % data_set)

Load app_store data set ...


In [13]:
dat_X_tok= tokenize(data_X.tolist())

In [17]:
dat_X_tok[i:i+5]

[Encoding(num_tokens=768, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=768, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=768, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=768, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=768, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])]

In [14]:
from tqdm import tqdm

In [20]:
embeddings = []
for i in tqdm(range(0, len(data_X), 100)):
    X = dict(tokenize(data_X[i:i+100].tolist()))
    embed = bert_encoder(X).numpy().tolist()
    embeddings.extend(embed)

100%|██████████| 64/64 [02:01<00:00,  1.90s/it]


In [32]:
s = bert_encoder(X)

2021-11-01 08:01:39.529919: W tensorflow/core/common_runtime/bfc_allocator.cc:457] Allocator (GPU_0_bfc) ran out of memory trying to allocate 14.04GiB (rounded to 15080620032)requested by op Tile
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2021-11-01 08:01:39.530021: I tensorflow/core/common_runtime/bfc_allocator.cc:1004] BFCAllocator dump for GPU_0_bfc
2021-11-01 08:01:39.530079: I tensorflow/core/common_runtime/bfc_allocator.cc:1011] Bin (256): 	Total Chunks: 13, Chunks in use: 11. 3.2KiB allocated for chunks. 2.8KiB in use in bin. 48B client-requested in use in bin.
2021-11-01 08:01:39.530090: I tensorflow/core/common_runtime/bfc_allocator.cc:1011] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2021-11-01 08:01:39.530099: I tensorflow/core/common_

ResourceExhaustedError: OOM when allocating tensor with shape[6392,768,768] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Tile]